# Fuentes

### Link: https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/
Explicación sobre las redes neuronales con múltiples capas, y lo que más interesante me pareció fue la enumeración de diferentes enfoques para saber cómo proponer la cantidad de capas y las neurones por cada capa a utilizar en el modelo de la red neuronal.

### Link: https://medium.com/fintechexplained/what-are-hidden-layers-4f54f7328263
En este artículo hace una explicación de las capas de una red neuronal de múltiples capas, útil para diferencias capa de entrada, salida y capas ocultas o intermedias.

### Link: https://mmuratarat.github.io/2019-06-12/embeddings-with-numeric-variables-Keras
Este artículo es importante porque te explica cómo usar la API funcional de Keras, que fue útil a la hora de utilizar embeddings porque necesitabamos que las nuevas entradas del modelo generadas por la capa de embedding se juntaran con las demás variables del problema, para lo cual tuvimos que emplear una capa *concatenate*, que funciona al utilizar la API funciona de Keras.


# 1. Cargando base de datos

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
import matplotlib.pyplot as plt

In [3]:
import numpy as np

In [4]:
import importlib

In [5]:
import sys

In [6]:
sys.path.insert(0, '..')

In [7]:
from src import helper
importlib.reload(helper);

In [8]:
import datetime

In [9]:
# Read the database from the .csv file into a pandas dataframe
df = pd.read_csv('../../databases/insurance.csv')

# 2. Preprocesamiento de los datos

In [10]:
from sklearn import preprocessing

## 2.1. Codificación de variables no numéricas o categóricas

In [11]:
# Create a label encoder for the sex variable or feature and create a new column in the dataframe 
# with the encoded version of the gender
sex_encoder = preprocessing.LabelEncoder()
sex_encoder.fit(df['sex'])
df['sex-encoded'] = sex_encoder.transform(df['sex'])

In [12]:
# Create a label encoder for the smoker variable or feature and create a new column in the dataframe
# with the encoded version of the smoker
smoker_encoder = preprocessing.LabelEncoder()
smoker_encoder.fit(df['smoker'])
df['smoker-encoded'] = smoker_encoder.transform(df['smoker'])

In [13]:
# Create a label encoder for the region variable or feature and create a new column in the dataframe
# with the encoded version of the region
region_encoder = preprocessing.LabelEncoder()
region_encoder.fit(df['region'])
df['region-encoded'] = region_encoder.transform(df['region'])

In [14]:
df.head()

,age,sex,bmi,children,smoker,region,charges,sex-encoded,smoker-encoded,region-encoded
0,19,female,27.900,0,yes,southwest,16884.92400,0,1,3
1,18,male,33.770,1,no,southeast,1725.55230,1,0,2
2,28,male,33.000,3,no,southeast,4449.46200,1,0,2
3,33,male,22.705,0,no,northwest,21984.47061,1,0,1
4,32,male,28.880,0,no,northwest,3866.85520,1,0,1


## 2.2. Filtrado o eliminación de variables no necesarias

In [72]:
# Filtering or removing of non desired variables
df_x = df[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded', 'region-encoded']]
df_y = df['charges']

# 3. Separación del conjunto de entrenamiento y evaluación

In [88]:
from sklearn import model_selection

In [89]:
from sklearn import preprocessing

## 3.1. Separación de los conjuntos
Es importante notar que, se realiza la separación del conjunto de datos original en **train**, **valid** y **test**, por fuera del framework de Keras para garantizar un adecuado tratamiento de los conjuntos acorde a la metodología empleada. En otras palabras, de esta forma nos aseguramos que cualquier preprocesamiento o normalización sobre validación (valid) y evaluación (test) se realiza a partir de la información obtenida en entrenamiento.

In [90]:
# Split the dataset into train_valid and test
x_train_valid, x_test, y_train_valid, y_test = model_selection.train_test_split(df_x, df_y, test_size=0.2, random_state=15, shuffle=True)

In [91]:
# Split the dataset into train and valid
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(x_train_valid, y_train_valid, test_size=0.3, random_state=23, shuffle=True)

## 3.2. Normalización de variables

In [92]:
# Select the variables where the z-score will be applied
scalable_variables = ['bmi', 'age']

if scalable_variables:
    # Create an instance of the StandardScaler for each variable
    scaler = preprocessing.StandardScaler()

    # Fit the distribution
    scaler.fit(x_train.loc[:, scalable_variables])

    # Transform and normalize all variables
    x_train.loc[:, scalable_variables] = scaler.transform(x_train.loc[:, scalable_variables])
    x_test.loc[:, scalable_variables] = scaler.transform(x_test.loc[:, scalable_variables])

## 3.3. Conjuntos de entrenamiento, validación y evaluación
Particularmente, para las redes neuronales de múltiples capas, en este problema vamos a estar utilizando una capa de *embedding* para poder reducir la dimensionalidad necesaria para desacoplar las categorías de la variable *region*. Es decir, normalmente podríamos utilizar un *one hot encoding* para independizar en diferentes variables, pero produce una mayor dimensionalidad y poca interpretabilidad de lo que la red neuronal aprende durante el entrenamiento. En conclusión, debemos separar los grupos de entradas por el formato de entrada de la red neuronal.

In [93]:
x_train = [x_train[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded']], x_train['region-encoded']]
x_valid = [x_valid[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded']], x_valid['region-encoded']]
x_test = [x_test[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded']], x_test['region-encoded']]

# 4. Multilayer Perceptron

In [78]:
from tensorflow import keras

In [79]:
from sklearn import metrics

In [80]:
tb_callback = keras.callbacks.TensorBoard(
    log_dir='tb-logs/mlp/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"), 
    histogram_freq=1,
    embeddings_freq=1, 
    update_freq='epoch'
)

In [81]:
def create_model(layers_neurons=[], layers_activation=[]):
    """ Creates a neural network model using the given hyperparameters, where the model is developed
        using the Keras framework. 
        The neural network is used to solve a regression problem where the input variables are:
            * bmi
            * sex
            * region
            * children
            * age 
            * smokes
        And the output variable is the amount of money charged for medical issues.
        
        @param layers_neurons List containing amount of neurons for each individual hidden layer
        @param layers_activation List containing the activation function for each individual hidden layer
    """

    # Validate the amount of layers described
    if len(layers_neurons) != len(layers_activation):
        raise ValueError('Invalid amount of layers in both the neurons and the activation function description')

    # Internal class parameters
    layer_count = len(layers_neurons)

    # Create two input layers for the categorical and the numerical variables
    x1 = keras.layers.Input(shape=(5, ))
    x2 = keras.layers.Input(shape=(1, ))

    # Create an embedding layer with the categorical variable and create
    # a new input layer for the neural network, combining both the embedding and the 
    # numerical variable input layer
    embedding = keras.layers.Embedding(4, 2, input_length=1, embeddings_initializer='normal')(x2)
    flatten = keras.layers.Flatten()(embedding)
    x = keras.layers.Concatenate()([x1, flatten])

    # Add the hidden layers to the neural network
    layers = []
    for layer_index in range(layer_count):
        layer_neurons = layers_neurons[layer_index]
        layer_activation = layers_activation[layer_index]
        previous_layer = layers[layer_index - 1] if layer_index else x
        current_layer = keras.layers.Dense(units=layer_neurons, activation=layer_activation)(previous_layer)
        layers.append(current_layer)

    # Add the output layer
    y = keras.layers.Dense(units=1, activation='linear')(current_layer)

    # Create the neural network model
    return keras.Model(inputs=[x1, x2], outputs=y)

In [99]:
def train_model(x_train, y_train, x_valid, y_valid, layers_neurons=[], layers_activation=[], lr=0.1, batch_size=64, epochs=100):
    """ Uses the create_model() function to create the neural network with the given hyperparameters,
        compiles the model using the corresponding optimizer, loss function, metrics and other hyperparameters,
        and runs the trainment process.
        
        @param x_train, y_train Train set
        @param x_valid, y_valid Valid set
        @param layers_neurons List containing amount of neurons for each individual hidden layer
        @param layers_activation List containing the activation function for each individual hidden layer
        @param lr Learning rate
        @param batch_size Batch size
        @param epochs Total amount of epochs for the training
    """
    
    # Create the neural network
    model = create_model(layers_neurons, layers_activation)
    model.summary()
    
    # Compile the neural network
    model.compile(
        optimizer=keras.optimizers.Adam(lr=lr),
        loss=keras.losses.MAE
    )
    
    # Train the neural network
    model.fit(
        x_train, y_train,
        validation_data=(x_valid, y_valid),
        epochs=epochs, verbose=1, shuffle=True, batch_size=batch_size,
        callbacks=[ tb_callback ]
    )

In [101]:
model = train_model(x_train, y_train, x_valid, y_valid, layers_neurons=[3, 3, 3], layers_activation=['relu', 'relu', 'relu'])
model.evaluate(x_test, y_test)

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 1, 2)         8           input_16[0][0]                   
__________________________________________________________________________________________________
input_15 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_5 (Flatten)             (None, 2)            0           embedding_7[0][0]                
____________________________________________________________________________________________

AttributeError: 'NoneType' object has no attribute 'evaluate'

In [ ]:
helper.plot_linear_regression_result(
    y_test.to_numpy(), 
    model.predict([x_test[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded']], x_test[['region-encoded']]]), 
    result_label='Costo atención médica'
)